<a href="https://colab.research.google.com/github/agmCorp/colab/blob/main/GenerateReport2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
# 1. Instalar la librería python-docx
!pip install python-docx

In [42]:
# 2. Subir el archivo 'Plantilla_Informe_Verificacion.docx' a Google Colab
#    (Esto debe hacerse manualmente a través del panel de archivos de Colab)

import re
from docx import Document
from docx.shared import Cm, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.document import Document as DocxDocument
from docx.text.paragraph import Paragraph
from docx.table import Table
from PIL import Image as PilImage, ImageDraw, ImageFont
import io
import os
from google.colab import files


In [45]:
def get_unavailable_image(width_px: int = 400, height_px: int = 300) -> io.BytesIO:
    """
    Crea una imagen PNG con el texto 'IMAGEN NO DISPONIBLE' en un búfer de memoria.

    :return: Un objeto io.BytesIO con la imagen en formato PNG.
    """
    # Usamos 400x300 para mantener el ratio original
    width_px = 400
    height_px = 300

    img = PilImage.new('RGB', (width_px, height_px), color='#E0E0E0') # Fondo gris claro
    draw = ImageDraw.Draw(img)

    font_size = 400
    try:
        font = ImageFont.truetype("arial.ttf", font_size)
    except IOError:
        font = ImageFont.load_default()

    text = "IMAGEN NO DISPONIBLE"

    # Obtener la caja delimitadora (bbox) para calcular el ancho y alto del texto
    left, top, right, bottom = draw.textbbox((0, 0), text, font)
    textwidth = right - left
    textheight = bottom - top

    # Calcular posición para centrar el texto
    x = (width_px - textwidth) / 2
    y = (height_px - textheight) / 2

    draw.text((x, y), text, fill=(50, 50, 50), font=font) # Texto gris oscuro

    # Guardar la imagen en un búfer de bytes en memoria (sin tocar el disco)
    img_byte_arr = io.BytesIO()
    img.save(img_byte_arr, format='PNG')
    img_byte_arr.seek(0) # Rebobinar el búfer al inicio

    return img_byte_arr

def generate_verification_report(template_path: str, output_path: str, replacements: dict, image_paths: dict):
    """
    Genera un informe DOCX a partir de una plantilla, reemplazando texto y
    insertando imágenes.
    """
    try:
        document = Document(template_path)
    except Exception as e:
        print(f"Error al cargar la plantilla {template_path}: {e}")
        return

    # Configuración de tamaño de imagen
    target_width_cm = 4.0
    target_width_emu = Cm(target_width_cm)

    def replace_in_element(element):
        """Reemplaza texto y placeholders de imagen en un párrafo o celda."""
        if isinstance(element, Paragraph):
            paragraphs = [element]
        elif isinstance(element, Table):
            paragraphs = [cell.paragraphs[0] for row in element.rows for cell in row.cells if cell.paragraphs]
        else:
            return

        for paragraph in paragraphs:
            full_text = "".join([run.text for run in paragraph.runs])
            image_inserted = False

            # 1. Reemplazo de imágenes (los placeholders {{..._image}})
            for img_placeholder, img_path in image_paths.items():
                token = f"{{{{{img_placeholder}}}}}"

                if token in full_text:
                    # Borrar el texto actual del párrafo antes de insertar la imagen/placeholder
                    for run in reversed(paragraph.runs):
                        paragraph._element.remove(run._element)

                    final_img_source = None # Contiene la ruta (str) o el búfer (io.BytesIO)
                    new_width_emu = target_width_emu

                    if img_path and os.path.exists(img_path):
                        # Caso 1: Imagen disponible y encontrada (Usa ruta de archivo)
                        final_img_source = img_path
                        try:
                            # Abrir la imagen para calcular sus dimensiones
                            img = PilImage.open(final_img_source)
                            original_width, original_height = img.size
                            aspect_ratio = original_height / original_width
                            new_height_emu = new_width_emu * aspect_ratio
                        except Exception as e:
                            # Si no puede abrir la imagen real, usa el placeholder de 'No disponible' (Usa búfer)
                            print(f"Advertencia: No se pudo abrir la imagen real '{img_path}'. Usando 'No Disponible' en memoria. Error: {e}")
                            final_img_source = get_unavailable_image() # Devuelve io.BytesIO
                            new_height_emu = target_width_emu * (300/400) # 400x300 ratio

                    else:
                        # Caso 2: Imagen NO disponible (Genera y usa búfer)
                        print(f"Generando placeholder 'No Disponible' para '{img_placeholder}' en memoria.")
                        final_img_source = get_unavailable_image() # Devuelve io.BytesIO
                        new_height_emu = target_width_emu * (300/400) # Usar 400x300 ratio

                    # Insertar la imagen (real (path) o placeholder (buffer))
                    if final_img_source:
                        run = paragraph.add_run()
                        # run.add_picture acepta tanto una ruta de archivo (str) como un objeto file-like (io.BytesIO)
                        run.add_picture(final_img_source, width=new_width_emu, height=new_height_emu)
                        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
                        image_inserted = True

                    break # Salir del bucle interno si se encontró el placeholder de imagen


            # 2. Reemplazo de texto (los placeholders {{...}} que quedan)
            if not image_inserted:
                full_text = "".join([run.text for run in paragraph.runs])
                new_text = full_text

                for placeholder, value in replacements.items():
                    new_text = new_text.replace(placeholder, str(value))

                if new_text != full_text:
                    for run in reversed(paragraph.runs):
                        paragraph._element.remove(run._element)
                    paragraph.add_run(new_text)


    # Recorrer Párrafos y Tablas
    for element in list(document.paragraphs) + list(document.tables):
        replace_in_element(element)

    # Guardar el nuevo documento
    document.save(output_path)

    print(f"\n✅ Informe generado con éxito: {output_path}")

In [46]:
# --- 1. Definir los reemplazos de texto ---
replacements = {
    # Datos Generales [cite: 3]
    "{{instance_id}}": "INS-20251010-001",
    "{{created_date}}": "2025/10/10",
    "{{created_time}}": "13:00:45",
    "{{requested_by_user_id}}": "user123",
    "{{requested_by_user_name}}": "Álvaro Morales",
    "{{requested_by_user_email}}": "alvaro.m@company.com",
    "{{requested_by_user_role}}": "Operator",
    "{{client_app_version}}": "1.5.2",
    "{{expected_prod_code}}": "PROD-ABC-456",
    "{{expected_prod_desc}}": "Paracetamol 500mg - Tabletas",

    # Producto [cite: 5]
    "{{expected_lot}}": "LOTE-2025-09-001",
    "{{validation_lot_ok}}": "✅",
    "{{expected_exp_date}}": "2027/12/31",
    "{{validation_exp_date_ok}}": "❌",
    "{{expected_pack_date}}": "2025/09/20",
    "{{validation_pack_date_ok}}": "❌",

    # Código de barras [cite: 7]
    "{{validation_barcode_detected_ok}}": "✅",
    "{{validation_barcode_legible_ok}}": "✅",
    "{{barcode_payload_decoded_value}}": "GS1-98765432101234",
    "{{barcode_payload_barcode_symbology}}": "DataMatrix",

    # Evidencias Visuales - Blob Info [cite: 9]
    "{{input_container}}": "cont-in-2025",
    "{{input_blob_name}}": "input_001.jpg",
    "{{processed_image_container}}": "cont-proc-2025",
    "{{processed_image_blob_name}}": "processed_001.jpg",
    "{{ocr_overlay_container}}": "cont-ocr-2025",
    "{{ocr_overlay_blob_name}}": "ocr_overlay_001.png",
    "{{barcode_overlay_container}}": "cont-bar-2025",
    "{{barcode_overlay_blob_name}}": "barcode_overlay_001.png",
    "{{barcode_roi_container}}": "cont-roi-2025",
    "{{barcode_roi_blob_name}}": "barcode_roi_001.png",

    # Resumen de Validaciones [cite: 21]
    "{{VALOR_AND}}": "✅",
    "{{validation_barcode_ok}}": "✅",
    "{{validation_summary}}": "✅",

    # Observaciones del Operario [cite: 23]
    "{{user_comment}}": "Producto verificado con éxito. Las fechas son visibles y legibles.",

    # Firma y Trazabilidad [cite: 25]
    "{{report_container}}": "cont-report-2025",
    "{{report_blob_name}}": "report_001.docx",
}


# --- 2. Crear imágenes de prueba (o simular la carga) ---
# En un entorno real, estas imágenes ya existirían y se cargarían.
# Aquí creamos imágenes temporales para simular el proceso.
temp_images = {
    "input_image.png": "red",
    "processed_image.png": "lightgray",
    "ocr_overlay_image.png": "lightblue",
    "barcode_overlay_image.png": "lightgreen",
    # Simular que la imagen barcode_roi_image NO ESTÁ DISPONIBLE
    #"barcode_roi_image.png": "yellow",
}

def create_dummy_image(filename, color):
    """Crea una imagen PNG de 400x300 de un color sólido para prueba."""
    img = PilImage.new('RGB', (400, 300), color=color)
    img.save(filename)
    return filename

print("Creando imágenes de prueba...")
for name, color in temp_images.items():
    create_dummy_image(name, color)
print("Imágenes de prueba creadas.")


# --- 3. Definir las rutas de las imágenes ---
# Las claves deben ser el nombre del placeholder SIN llaves (e.g., 'input_image')
image_paths = {
    "input_image": "input_image.png",
    "processed_image": "processed_image.png",
    "ocr_overlay_image": "ocr_overlay_image.png",
    "barcode_overlay_image": "barcode_overlay_image.png",
    # Se pasa None o una cadena vacía para simular que no está disponible
    "barcode_roi_image": None,
}

# --- 4. Ejecución de la rutina ---
TEMPLATE_NAME = "/content/sample_data/Plantilla_Informe_Verificacion.docx"
OUTPUT_NAME = "/content/sample_data/Plantilla_Informe_Verificacion_final.docx"

# La plantilla debe estar subida a Colab para que esta línea funcione
generate_verification_report(TEMPLATE_NAME, OUTPUT_NAME, replacements, image_paths)

# --- 5. Descargar el archivo generado (Opcional en Colab) ---
try:
    files.download(OUTPUT_NAME)
    print(f"\nDescargando {OUTPUT_NAME}...")
except Exception as e:
    print(f"\nError al descargar el archivo: {e}")

Creando imágenes de prueba...
Imágenes de prueba creadas.
Generando placeholder 'No Disponible' para 'barcode_roi_image' en memoria.

✅ Informe generado con éxito: /content/sample_data/Plantilla_Informe_Verificacion_final.docx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Descargando /content/sample_data/Plantilla_Informe_Verificacion_final.docx...
